In [4]:
using Luxor
using IntervalSets
using ForwardDiff
using Plots
gr()

function Bsp(p,k,i::Int64,t)::Float64
    if(p==0)
        return k[i]≤t<k[i+1]||(k[i]≠k[i+1]==k[end]==t)
    else
        B=0;
        if(k[i+p]-k[i]≠0)
            B+=Bsp(p-1,k,i,t)*(t-k[i])/(k[i+p]-k[i])
        end
        if(k[i+p+1]-k[i+1]≠0)
            B+=Bsp(p-1,k,i+1,t)*(k[i+p+1]-t)/(k[i+p+1]-k[i+1])
        end
        return B
    end
end

function cof(f,a,b)
    f(a),
    3*(f(2*a/3+b/3)-(8*f(a)+f(b))/27)-3*(f(a/3+2*b/3)-(f(a)+8*f(b))/27)/2,
    -3*(f(2*a/3+b/3)-(8*f(a)+f(b))/27)/2+3*(f(a/3+2*b/3)-(f(a)+8*f(b))/27),
    f(b)
end

function fastuniq(v)
  v1 = Vector{eltype(v)}()
  if length(v)>0
    laste = v[1]
    push!(v1,laste)
    for e in v
      if !(e ≈ laste)
        laste = e
        push!(v1,laste)
      end
    end
  end
  return v1
end

function Mesh(k,partn)
    K=fastuniq(k)
    K=[K[i]+(j/(partn))*(K[i+1]-K[i]) for i in 1:(length(K)-1), j in 0:(partn-1)]
    append!(sort(reshape(K,prod(size(K)))),k[end])
end

function BspSurface(N;name="NURBSmfd.svg",up=5,down=-5,right=5,left=-5,step=50)
    d,p,k,w,a=N
    p₁,p₂=p
    k₁,k₂=k
    n₁,n₂=length(k₁)-p₁-1,length(k₂)-p₂-1
    P(t)=sum(w[i₁,i₂]*Bsp(p₁,k₁,i₁,t[1])*Bsp(p₂,k₂,i₂,t[2])*a[i₁,i₂,:]/sum(w[j₁,j₂]Bsp(p₁,k₁,j₁,t[1])*Bsp(p₂,k₂,j₂,t[2]) for j₁ in 1:n₁,j₂ in 1:n₂) for i₁ in 1:n₁,i₂ in 1:n₂)

    Drawing(step*(right-left),step*(up-down),name)
    Luxor.origin(-step*left,step*up)
    background("white")
    sethue("red")

    K₁=fastuniq(k₁[1+p₁:end-p₁])
    K₂=fastuniq(k₂[1+p₂:end-p₂])
    if((length(fastuniq(w))>1)||(p₁>3)||(p₂>3))
        K₁=Mesh(K₁,10)
        K₂=Mesh(K₂,10)
    end
    N₁=length(K₁)
    N₂=length(K₂)
    m₁=10
    m₂=10
    
    for t₁ in linspace(K₁[1],K₁[end],m₁+1)
        move(step*Point([1,-1].*P([t₁,K₂[1]])...))
        for i in 1:(N₂-1)
            a₁,a₂,a₃,a₄=cof(t->P([t₁,t]),K₂[i],K₂[i+1])
            curve(step*Point([1,-1].*a₂...),step*Point([1,-1].*a₃...),step*Point([1,-1].*a₄...))
        end
        strokepath()
    end

    for t₂ in linspace(K₂[1],K₂[end],m₂+1)
        move(step*Point([1,-1].*P([K₁[1],t₂])...))
        for i in 1:(N₁-1)
            a₁,a₂,a₃,a₄=cof(t->P([t,t₂]),K₁[i],K₁[i+1])
            curve(step*Point([1,-1].*a₂...),step*Point([1,-1].*a₃...),step*Point([1,-1].*a₄...))
        end
        strokepath()
    end

    sethue("black")
    setline(1)
    Cp=[step*Point([1,-1].*a[i,j,:]...) for i in 1:size(a)[1], j in 1:size(a)[2]]
    map(p->circle(p,2,:fill), Cp)
    for i in 1:size(a)[1]
        poly(Cp[i,:], :stroke)
    end
    for j in 1:size(a)[2]
        poly(Cp[:,j], :stroke)
    end
    
    finish()
end

function href(N,k₊)
    (d,p,k,w,a)=N
    n=[length(k[l])-p[l]-1 for l in 1:d]
    pᵣ=p
    kᵣ=[sort(vcat(k[l],k₊[l])) for l in 1:d]
    nᵣ=[length(kᵣ[l])-pᵣ[l]-1 for l in 1:d]
    κ=[[((nᵣ[l]+1-i)/(nᵣ[l]+1))*kᵣ[l][i]+((i)/(nᵣ[l]+1))*kᵣ[l][i+pᵣ[l]+1] for i in 1:nᵣ[l]] for l in 1:d]
    C=[[Bsp(pᵣ[l],kᵣ[l],i,κ[l][j]) for j in 1:nᵣ[l], i in 1:nᵣ[l]] for l in 1:d]
    D=[C[l]\[Bsp(p[l],k[l],j,κ[l][i]) for i in 1:nᵣ[l], j in 1:length(k[l])-p[l]-1] for l in 1:d]
    wᵣ=[sum(w[i₁,i₂]*D[1][l₁,i₁]*D[2][l₂,i₂] for i₁ in 1:length(k[1])-p[1]-1, i₂ in 1:length(k[2])-p[2]-1) for l₁ in 1:nᵣ[1], l₂ in 1:nᵣ[2]]
    aᵣ=[sum(w[i₁,i₂]*D[1][l₁,i₁]*D[2][l₂,i₂]*a[i₁,i₂,j] for i₁ in 1:length(k[1])-p[1]-1, i₂ in 1:length(k[2])-p[2]-1) for l₁ in 1:nᵣ[1], l₂ in 1:nᵣ[2], j in 1:2]./wᵣ
    Nᵣ=(d,pᵣ,kᵣ,wᵣ,aᵣ)
end

function pref(N,p₊)
    (d,p,k,w,a)=N
    n=[length(k[l])-p[l]-1 for l in 1:d]
    pᵣ=p+p₊
    k₊=[repeat(fastuniq(k[l]),inner=p₊[l]) for l in 1:d]
    kᵣ=[sort(vcat(k[l],k₊[l])) for l in 1:d]
    nᵣ=[length(kᵣ[l])-pᵣ[l]-1 for l in 1:d]
    κ=[[((nᵣ[l]+1-i)/(nᵣ[l]+1))*kᵣ[l][i]+((i)/(nᵣ[l]+1))*kᵣ[l][i+pᵣ[l]+1] for i in 1:nᵣ[l]] for l in 1:d]
    C=[[Bsp(pᵣ[l],kᵣ[l],i,κ[l][j]) for j in 1:nᵣ[l], i in 1:nᵣ[l]] for l in 1:d]
    D=[C[l]\[Bsp(p[l],k[l],j,κ[l][i]) for i in 1:nᵣ[l], j in 1:length(k[l])-p[l]-1] for l in 1:d]
    wᵣ=[sum(w[i₁,i₂]*D[1][l₁,i₁]*D[2][l₂,i₂] for i₁ in 1:length(k[1])-p[1]-1, i₂ in 1:length(k[2])-p[2]-1) for l₁ in 1:nᵣ[1], l₂ in 1:nᵣ[2]]
    aᵣ=[sum(w[i₁,i₂]*D[1][l₁,i₁]*D[2][l₂,i₂]*a[i₁,i₂,j] for i₁ in 1:length(k[1])-p[1]-1, i₂ in 1:length(k[2])-p[2]-1) for l₁ in 1:nᵣ[1], l₂ in 1:nᵣ[2], j in 1:2]./wᵣ
    Nᵣ=(d,pᵣ,kᵣ,wᵣ,aᵣ)
end

pref (generic function with 1 method)

In [7]:
d=2
p=[2,3]
k=[[0,0,0,1,2,5,5,5],[0,0,0,0,7,7,7,7]]
w=[1 for i in -2:2, j in -2:1]
a=[j*k+i*(1-k) for i in -2:2, j in -2:1, k in [0,1]]+(rand(5,4,2)-1/2)*0.9
N=(d,p,k,w,a)

(2, [2, 3], Array{Int64,1}[[0, 0, 0, 1, 2, 5, 5, 5], [0, 0, 0, 0, 7, 7, 7, 7]], [1 1 1 1; 1 1 1 1; … ; 1 1 1 1; 1 1 1 1], [-2.24873 -1.83294 -1.95363 -2.02465; -1.06937 -0.711976 -0.780874 -1.32435; … ; 1.26335 1.32781 0.798915 1.44142; 2.31281 2.23476 1.8539 1.95432]

[-1.96938 -1.20529 0.43931 1.26382; -2.23539 -0.660634 0.374967 0.58342; … ; -2.12239 -1.3856 -0.19823 1.40417; -2.00616 -1.39238 -0.420149 1.1918])

In [8]:
BspSurface(N,name="2dim.svg")
k₊=[[0.5,1.2,4,4.5],[]];
BspSurface(href(N,k₊),name="2dimh.svg")
p₊=[2,3];
BspSurface(pref(N,p₊),name="2dimp.svg")

[0, 0, 1, 1, 2, 2, 5, 5]
Array{Int64,1}[[0, 0, 1, 1, 2, 2, 5, 5], [0, 0, 0, 7, 7, 7]]

true